# Import Libraries

Run the following code on terminal:

/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip

/usr/local/Cellar/jupyterlab/3.0.14/libexec/bin/python3.9 -m pip install pandas <br>
/usr/local/Cellar/jupyterlab/3.0.14/libexec/bin/python3.9 -m pip install numpy <br>
/usr/local/Cellar/jupyterlab/3.0.14/libexec/bin/python3.9 -m pip install pd <br>
pip3 install beautifulsoup4 <br>
pip3 install selenium <br>
pip install webdriver-manager  <br>
sudo pip install lxml


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import re #regex
import lxml #use html.parser instead / no dependency issues

# Scrape the Data

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

# Load the web page
url = "https://www.tjk.org/TR/YarisSever/Info/Page/GunlukYarisProgrami"
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')

data2 = []
race_metadata = []
ids = []
all_data = []

#### SELECT A DATE RANGE 
#Data is availabe after 01/01/2018

date_generated = pd.date_range(end = datetime.today(), periods = 5) #periods = 3 can use instead of start start='1/1/2018'
date_range = date_generated.strftime("%d/%m/%y").tolist()

for date in date_range:

    elem = driver.find_element_by_name('QueryParameter_Tarih')
    elem.clear()
    elem.send_keys(date)
    time.sleep(20) #MAYBE WAIT LESS?? Don't wait less. In 10 s: Sayfayı yüklemeden şehirleri aldığı için
    #saydfa yüklendiğinde şehir adları eşleşmediğinde unavailable path hatası verdi(09/02/2021)

    
    ## LOOP THROUGH THE ALL TABS
    
    ids.clear()
    # For loop that iterates over all the <li> tags
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for h in soup.findAll('li'):

        # looking for anchor tag inside the <li>tag
        a = h.find('a')
        try:

            # looking for href inside anchor tag
            if 'data-sehir-id' in a.attrs:

                # storing the value of href in a separate 
                # variable
                city_id = a.get('id')

                # appending the url to the output list
                ids.append(city_id)

        # if the list does not has a anchor tag or an anchor 
        # tag does not has a href params we pass
        except:
            pass   
        
    for city in ids: 
    
        elem1= driver.find_element_by_xpath("//ul[@class='gunluk-tabs']")

        xpath = "//*[@id='"
        xpath += str(city)
        xpath += "']"
        driver.find_element_by_xpath(xpath).click()
        time.sleep(10)
    
    
    ## COLLECT ALL THE RACES METADATA

        # find all elements inside a div element of class col-lg-10
        selector = 'div.race-details > *'

        # find elements that contain the data we want
        found = soup.select(selector)

        # Extract data from the found elements
        data = [x.text.split(';')[-1].strip() for x in found]

        #remove unnecessary characters and spaces
        for element in data:
            data2.append(" ".join(element.split()))
        N = 2
        # Consecutive N concatenation in String list
        # using format() + zip() + iter() + list comprehension
        temp = '{} ' * N 
        race_metadata_list = [temp.format(*ele) for ele in zip(*[iter(data2)] * N)]
        race_metadata.extend(race_metadata_list)
    
        
     ## COLLECT RUN/RACE DETAILED DATA/RESULTS
            
        tables = soup.find_all('table')
        dfs = pd.read_html(str(tables))
        
    
     ## PİST METADATA // Check out sentetik again in 09.06.2021, there was a problem but it may be related to 
        #the set_index function
        
        sentetik = soup.find_all("span", {"class": "raceWeatherBrown"})
        sentetik = [x.text.split(' ')[-1].strip() for x in sentetik]
        
        çim = soup.find_all("span", {"class": "raceWeatherGreen"})
        çim = sentetik = [x.text.split(' ')[-1].strip() for x in çim]
        
        conditions = çim+sentetik
        
        str_conditions = ";".join(conditions)
            
     ## COMBINE ALL_DATA 
    
        for x, df in enumerate(dfs):
            df = dfs[x]
            df['tarih']= date
            df['lokasyon']= city
            df['metadata'] = race_metadata_list[x]
            df['race-conditions'] =  str_conditions
            
            #For sequences, (strings, lists, tuples), use the fact that empty sequences are false
            all_data.append(df)
            


In [ ]:
driver.quit()

# Data Understanding

In [ ]:
len(all_data) #number of races/ extracted race tables

In [ ]:
all_data[5].tarih

In [ ]:
#For each race, assign a new column to indicate S/F Horse 
 #For each race, generate a unique id
##DONT RUN
#A Unique ID
#for df in all_data:
 #   for x in df:
  #      df[x] = "x"
        
#Winning Type
#if En iyi D. & includes (1) in AGF = Favori, else Sürpriz

In [ ]:
df = pd.concat(all_data)
df.head(-10)

In [ ]:
df['metadata'][1]

In [ ]:
# new data frame with split value columns
new = df["metadata"].str.split(",", n = 5, expand = True)
new #length varies

In [ ]:
metadata = pd.DataFrame({'race' : []})

metadata["race"] = new[0]
metadata["group"] = new[1]
conditions = [
        (pd.notnull(new[5]) & pd.notnull(new[4])),
        (pd.isnull(new[5]) & pd.notnull(new[4])),
        (pd.isnull(new[5]) & pd.isnull(new[4]))]

choices_kg = [new[2]+new[3], new[2], None]
metadata['kg'] = np.select(conditions, choices_kg, default=None)

choices_eid = [new[5], new[4], new[3]]
metadata['eid'] = np.select(conditions, choices_eid, default=None) #Koşunun EİD'si/ metadata yarışla alakalı

choices_pist = [new[4], new[3], new[2]]
metadata['pist'] = np.select(conditions, choices_pist, default=None)

metadata.head(5)


In [ ]:
#Append metadata to df 
df_1 = pd.concat([df, metadata], axis=1)
df_1.head()
#DF1 includes metadata splitted 
#DF2 includes metadata extended by AGF/ratio/F/S; hour, race no, tür, kg by numeric, eid by numeric, mesafe and pist türü


#TOBE MERGED WITH JOKEY FEATURES; SAHIP & ANTRENOR FEATURES; 

In [ ]:
x = df_1['pist'].str.extract(r"(\d{4})", expand=True)[0]


In [ ]:
#REGEX OPS

#Extract time from HH/MM format
df['Time']=df_1['race'].str.extract(r"(([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9])", expand=True)[0]
#Extract Tür after HH/MM format
 
#Extract n_koşu before HH/MM format
df['gunluk_kosu_no']=df_1['race'].str.extract(r"(^.*?(?=:([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]))")[0]
#Pist mesafe & type
df['Mesafe']=df_1['pist'].str.extract(r"(\d{4})", expand=True)[0]
#extract eid as numeric

#extract kg as numeric

#orijin split anne-baba by "-" till /

#orijin afte r / lokasyon/doğum yeri belki?

#at yaşı numeric

#at yaşı #y sonrası

#siklet +? eklenen puan ayrı yeni bir sütun

#KG as binary may be better like 58=1 59=1, extract digits by regex
#Seperate anne-baba 
#split yaş ve grup

#AGF Favourite/Surprise (favourite can be used as a feature while suprise can be calculated maybe?)


In [ ]:
df['Kosu_Türü']=df['n_kosu'].str.extract(r"?<=([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9].*", expand=True)[0]

In [ ]:
df.drop(columns=["pist_1", "mesafe"])
df

In [ ]:
###EXPORT DATASET TO CSV
all_data.to_csv('yaris_detayli')

## Keep running the code / try again after sleep

'NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='Şanlıurfa']"}'

```python
import time
 
def do_work():
    # whatever you're doing
 
while True:
    try:
        do_work()
    except:
        time.sleep(10)
```
